In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
from utils.qwen import extract_article_metadata 
import pandas as pd
import tqdm
tqdm.tqdm.pandas()

In [3]:
data = pd.read_csv("03-pages_cleaned.csv")

In [5]:
data.head(1)

,article_id,title,abstract_ar,authors,authors_en,general_field,publish_date,page1,page2,source,path,keywords
0,1,درجة اكتساب طلبة كلية التربية بجامعة السلطان ق...,NaN,"[""نور النجار""]",NaN,NaN,NaN,Psychology\nVolume 23 Issue 2 Article 1\nلمهار...,مجله اتحاد الجامعات العربيه للتربيه وعلم النفس...,AJP,finalpdfs/AJP_1337_1.pdf,None


### expermient with sample 

In [6]:
sample = data.sample(1).iloc[0]

In [7]:
type(sample)

pandas.core.series.Series

In [8]:
sample[["article_id","path"]]

article_id                  310
path          finalpdfs/194.pdf
Name: 300, dtype: object

In [9]:
page1,page2 = sample["page1"] ,sample["page2"]

In [10]:
from IPython.display import HTML, display

display(HTML(f"""
<div dir="rtl" style="text-align: right; font-size: 16px;">
page1 <br>
{page1}
<br>
page2 <br>
{page2}
<br>
</div>
"""))

In [11]:
json=extract_article_metadata(page1,page2)

In [12]:
from IPython.display import HTML, display

display(HTML(f"""
<div dir="rtl" style="text-align: right; font-size: 16px;">
{json}
</div>
"""))

In [13]:
json.keys() 

dict_keys(['title', 'abstract_ar', 'general_field', 'authors', 'keywords', 'publish_date'])

In [14]:
sample.keys()

Index(['article_id', 'title', 'abstract_ar', 'authors', 'authors_en',
       'general_field', 'publish_date', 'page1', 'page2', 'source', 'path',
       'keywords'],
      dtype='object')

## process row function

In [17]:
# this is a  function that takes a row and returns a new row processed 
from utils.processing import process_row

processed = process_row(sample)

In [23]:
display(HTML(f"""
<div dir="rtl" style="text-align: right; font-size: 16px;">
{processed.to_dict()}
</div>
"""))

### now we need just to parallelise it 

In [80]:
# let's get a csv sample tp simulate processing 
subset = data.sample(2)
subset

,article_id,title,abstract_ar,authors,authors_en,general_field,publish_date,page1,page2,source,path,keywords
1270,1282,NaN,NaN,NaN,NaN,Business,NaN,دافعيه الانجاز وعلاقتها باساليب مواجهه الضغوط\...,دافعيه الانجاز وعلاقتها باساليب مواجهه الضغوط ...,ARPD,finalpdfs/47.pdf,None
320,330,NaN,NaN,NaN,NaN,Agricultural,NaN,المجله الاردنيه في العلوم التربويه. مجلد 16 st...,المجله الاردنيه في العلوم التربويه\n2- يحاول ا...,ARPD,finalpdfs/211.pdf,None


In [81]:
subset = subset.progress_apply(process_row,axis=1)

100%|██████████████████████████████████████████████████| 2/2 [05:16<00:00, 158.25s/it]


In [36]:
subset

,article_id,title,abstract_ar,authors,authors_en,general_field,publish_date,page1,page2,source,path,keywords
76,86,التسويق الالكتروني لخدمات المعلومات فا المكتبا...,هدفت الدراسه الا الكشف عن واقع التسويق الالكتر...,[جمال الحباري],"[""Samar Gamal Al-Habbari""]",Education,2023-01-01,Arab Journal of Sciences Research Publishing (...,E-Marketing of Information Services in Yemeni ...,ajsrp,finalpdfs/6522-6170.pdf,"[المكتبات الجامعيه, التسويق الالكتروني, خدمات ..."
363,373,اثر التدريب المبدانا,هدفت هذه الدراسه الا فحص اثر خبره التدريب المي...,[محمد الخوالده],NaN,Agricultural,2018-01-01,اثر التدريب المبدانا\noes 4ه ب\nلدا طلبه\n3\nم...,المجله الاردنيه في العلوم التربويه\nان التدريب...,ARPD,finalpdfs/250.pdf,"[التدريب الميداني, كفايات الارشاد المهني, الرض..."


### Names also need a normalisation process but not now 

In [79]:
subset.to_csv("sample.csv")